<a href="https://colab.research.google.com/github/david-j-cox/Man-vs-Machine/blob/RE288/GME_fits_re24_re288.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
# Set working directory
from google.colab import drive
drive.mount('/content/gdrive')
%cd './gdrive/My Drive/GME & MLB/Scripts and Notebooks/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/GME & MLB/Scripts and Notebooks


In [ ]:
# Connect GPU, if desired
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [2]:
# Connect TPU, if desired
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.7.0
Running on TPU  ['10.4.167.34:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.4.167.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.4.167.34:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
# Packages and modules we'll use
# System
import glob
import zipfile, io, os
import time
from IPython.display import HTML, display, clear_output

# Data manipulations
import pandas as pd
import numpy as np
import math

print("Packages loaded")

Packages loaded


# Fit the GME for each pitcher in the df using specified SR+

### Caution: This will run for a long time. Break into batches if possible. 

In [5]:
# Create lists of which pitches fall into which categories
hrd_strt = ['4-Seam Fastball', '2-Seam Fastball', 'Cutter', 'Pitch Out']
hrd_brkng = ['Slider', 'Sinker', 'Split Finger', 'Screwball', 'Forkball']
sft_strt = ['Changeup', 'Knuckle Ball', 'Eephus']
sft_brkng = ['Curveball', 'Knuckle Curve', 'Unknown']

In [ ]:
# Compute matching for each df across all years
for yr in [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]:
  # Read in the target dataframe
  data = pd.read_csv(f'../Data/03_analytic/df_{yr}.csv').drop(['Unnamed: 0'], axis=1)

# Lists for storing results
  year = []
  pitcher_id = []
  reinforcer = []
  vac = []
  bias = []
  sensitivity = []

  # For reporting prgress to notebook user
  count = 0 

  # Loop through dataframe, fit GME, and get fit metrics for every pitcher
  uniq_p = data['pitcher'].unique()
  for pitcher in uniq_p:
    # For each putative reinforcer we're trying...
    for sr_type in ['re_24_change_raw', 're_24_change_ord', 're_288_change_raw', 're_288_change_ord']:
      # Empty lists to store counts in
      # Behaviors
      count_hard_straight = []
      count_hard_breaking = []
      count_soft_straight = []
      count_soft_breaking = []
      # Consequences
      count_sr_hs = []
      count_sr_hb = []
      count_sr_ss = []
      count_sr_sb = []

      # Isolating the pitcher's data for the season
      temp_df = data[data['pitcher']==pitcher]

      # Get list of unique games in which they pitched. Skip if pitched in less than 3 games
      uniq_games = temp_df['game_pk'].unique()
      if len(uniq_games) < 3:
        continue
      else:
        # Isolate each unique game 
        for game in uniq_games:
          game_data = temp_df[temp_df['game_pk']==game]
          # Create dfs for each pitch type
          hard_straight = game_data.loc[game_data['pitch_name'].isin(hrd_strt)]
          hard_breaking = game_data.loc[game_data['pitch_name'].isin(hrd_brkng)]
          soft_straight = game_data.loc[game_data['pitch_name'].isin(sft_strt)]
          soft_breaking = game_data.loc[game_data['pitch_name'].isin(sft_brkng)]
          
          # Get the count of each pitch type per game
          count_hard_straight.append(len(hard_straight))
          count_hard_breaking.append(len(hard_breaking))
          count_soft_straight.append(len(soft_straight))
          count_soft_breaking.append(len(soft_breaking))

          # Get the count of putative reinforcers that followed each pitch type per game
          count_sr_hs.append(hard_straight[hard_straight[sr_type]>0][sr_type].sum())
          count_sr_hb.append(hard_breaking[hard_breaking[sr_type]>0][sr_type].sum())
          count_sr_ss.append(soft_straight[soft_straight[sr_type]>0][sr_type].sum())
          count_sr_sb.append(soft_breaking[soft_breaking[sr_type]>0][sr_type].sum())

      # Log pitch and outcome ratios using fastball as Bi
      logged_bx = []
      logged_strikes = []
      for l in list(range(len(count_hard_straight))):
        behavior = (count_hard_straight[l]+1)/(count_hard_breaking[l] + count_soft_straight[l] + count_soft_breaking[l]+1)
        strikes = (count_sr_hs[l]+1)/(count_sr_hb[l] + count_sr_ss[l] + count_sr_sb[l]+1)
        logged_bx.append(math.log(behavior, 10))
        logged_strikes.append(math.log(strikes, 10))
      count_data = pd.DataFrame({'Logged_Bx':logged_bx, 'Logged_Strikes':logged_strikes})
      count_data.fillna(0)
      
      # Fit the GME and return the parameters
      if len(count_data)>0:
        s_str, b_str = np.polyfit(count_data['Logged_Strikes'], count_data['Logged_Bx'], 1)
        corrmatSTR = np.corrcoef(count_data['Logged_Strikes'], count_data['Logged_Bx'])
        r2_str = (corrmatSTR[0,1])**2

        # Append matching params
        year.append(yr)
        pitcher_id.append(pitcher)
        reinforcer.append(sr_type)
        vac.append(round(r2_str, 4))
        bias.append(round(b_str, 4))
        sensitivity.append(round(s_str, 4))
      else:
        continue
    
    # After every 5th player completed in a season...
    # Save data
    player_fits = pd.DataFrame({'Player':pitcher_id,
                                'Season':year,
                                'Reinforcer': reinforcer, 
                                'VAC':vac,
                                'Bias':bias,
                                'Sensitivity':sensitivity,
                                })
    player_fits.to_csv(f'../Data/06_model_output/gme_fits_{yr}.csv')

    # Provide notebook user with update on progress 
    count +=1
    if count%5==0:
      clear_output()
      print(f"Working on {yr}\n{count} completed out of {len(uniq_p)}. ({round(100*(count/len(uniq_p)), 2)}%)")

Working on 2008
505 completed out of 630. (80.16%)
